The codes are modified in object_detection_tutorial.ipynb of the TensorFlow Object Detection API. 

You can see the original version of the codes in the TensorFlow Object Detection API. 
https://github.com/tensorflow/models/tree/master/research/object_detection

The files is edited for detection of the acrosome reaction of sperm by Dr. Sang-Hee Lee. 
If you have question regarding codes and information, please send the email.

Email: knusang1799@gmail.com

Additionally, you can download acrosome reaction classification system (ARCS) containing trained models based on ResNet 50 and Inception-ResNet v2. 

Github: https://github.com/knusang1799/acrosome-reaction-classification-system.git

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import glob


from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from datetime import datetime


# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops # SH finish

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')


/home/sanghee/anaconda3/envs/tensorflow1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sanghee/anaconda3/envs/tensorflow1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sanghee/anaconda3/envs/tensorflow1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sanghee/anaconda3/envs

## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

In [3]:
from utils import label_map_util # SH finish
from utils import visualization_utils2_AR as vis_util

In [4]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Model preparation 

In [14]:
MODEL_NAME_DIR = 'models'

# Please select the proper model

MODEL_NAME = '/ResNet-400-mag.pb'
# MODEL_NAME = '/ResNet-1000-mag.pb'
# MODEL_NAME = '/Incep-Res-400-mag.pb'
# MODEL_NAME = '/Incep-Res-1000-mag.pb'

LABEL_NAME = '/AR_labelmap.pbtxt'

PATH_TO_FROZEN_GRAPH = MODEL_NAME_DIR + MODEL_NAME
PATH_TO_LABELS = MODEL_NAME_DIR + LABEL_NAME

In [15]:
# MODEL_NAME[1:-3]

## Load a (frozen) Tensorflow model into memory.

In [16]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [17]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper code

In [18]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [20]:
TEST_IMAGE_PATHS = glob.glob('test_dataset/*.JPG', recursive=True)
# IMAGE_SIZE = (12, 8)

In [ ]:
# if MODEL_NAME == '/ResNet-400-mag.pb':
#     TEST_IMAGE_PATHS = glob.glob('test_dataset/400_mag_*.JPG', recursive=True)
    
# elif MODEL_NAME == '/Incep-Res-400-mag.pb':
#     TEST_IMAGE_PATHS = glob.glob('test_dataset/400_mag_*.JPG', recursive=True)
    
# elif MODEL_NAME == '/ResNet-1000-mag.pb':
#     TEST_IMAGE_PATHS = glob.glob('test_dataset/1000_mag_*.JPG', recursive=True)
    
# elif MODEL_NAME == '/Incep-Res-1000-mag':
#     TEST_IMAGE_PATHS = glob.glob('test_dataset/1000_mag_*.JPG', recursive=True)

In [21]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [22]:
MIN_SCORE_THRESH = .80

In [23]:
for image_path in TEST_IMAGE_PATHS:
        
    image = Image.open(image_path)
    print("exporting....:",os.path.basename(image_path))
    print()
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
    output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
  # Visualization of the results of a detection.
    a = vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,
          instance_masks=output_dict.get('detection_masks'),
          use_normalized_coordinates=True,
          # line_thickness=4, 
          line_thickness=3, 
          min_score_thresh=MIN_SCORE_THRESH
          )

    img = Image.fromarray(image_np)
    img.save('evaluated_datasets' + image_path[12:-4]+'_'+MODEL_NAME[1:-3]+'.png')

exporting....: 400_mag_testset_1.JPG

exporting....: 400_mag_testset_2.JPG

exporting....: 400_mag_testset_3.JPG

exporting....: 400_mag_testset_4.JPG

exporting....: 400_mag_testset_5.JPG

